In [0]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from sklearn.cluster import KMeans

In [7]:
from google.colab import files
uploaded = files.upload()

Saving export2009.csv to export2009 (1).csv
Saving export2010.csv to export2010 (1).csv
Saving export2011.csv to export2011 (1).csv
Saving export2012.csv to export2012 (1).csv
Saving export2013.csv to export2013 (1).csv
Saving export2014.csv to export2014 (1).csv
Saving export2015.csv to export2015 (1).csv
Saving export2016.csv to export2016 (1).csv
Saving export2017.csv to export2017 (1).csv
Saving export2018.csv to export2018 (1).csv


# 1. **Data Processing**
### **1.1 Download the dataset for the period 2009 – 2018 and create a dataframe that concatenates Wales (total) employment value only.**

In [0]:
df2009 = pd.read_csv(io.BytesIO(uploaded['export2009.csv']))
df2010 = pd.read_csv(io.BytesIO(uploaded['export2010.csv']))
df2011 = pd.read_csv(io.BytesIO(uploaded['export2011.csv']))
df2012 = pd.read_csv(io.BytesIO(uploaded['export2012.csv']))
df2013 = pd.read_csv(io.BytesIO(uploaded['export2013.csv']))
df2014 = pd.read_csv(io.BytesIO(uploaded['export2014.csv']))
df2015 = pd.read_csv(io.BytesIO(uploaded['export2015.csv']))
df2016 = pd.read_csv(io.BytesIO(uploaded['export2016.csv']))
df2017 = pd.read_csv(io.BytesIO(uploaded['export2017.csv']))
df2018 = pd.read_csv(io.BytesIO(uploaded['export2018.csv']))

In [9]:
df = df2009[['Wales']]
df.columns = ['2009']
df['2010'] = df2010['Wales']
df['2011'] = df2011['Wales']
df['2012'] = df2012['Wales']
df['2013'] = df2013['Wales']
df['2014'] = df2014['Wales']
df['2015'] = df2015['Wales']
df['2016'] = df2016['Wales']
df['2017'] = df2017['Wales']
df['2018'] = df2018['Wales']
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,37700,38200,36100,36100,36800,42700,40700,43200,40200,41100
1,156700,149800,158600,154400,164200,173300,172300,162500,165100,165700
2,96600,93200,90000,91300,89300,97000,92600,102700,90800,101800
3,345400,344500,343100,347300,345100,337300,357700,360200,333500,347600
4,27800,27900,26400,27200,26900,35700,24000,34400,58900,31500
5,33800,29800,33200,31100,32400,32400,30800,31000,32100,35500
6,13500,14600,17600,18800,18000,22200,19100,22700,18200,25200
7,144800,145800,143600,137300,149900,152900,166200,161200,176400,187100
8,415600,418600,425600,421000,427000,427600,423200,418500,424500,434900
9,64200,68000,72400,72800,75500,73300,77200,72400,83200,81800


### **1.2 Check for any null value or outlier. If found replace that with mean value.**

In [10]:
df.isna().sum()

2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    0
dtype: int64

No null value found. All values are present.

### **1.3 Change the name of the industries**





In [11]:
df.index = ['Agriculture', 'Production', 'Construction', 'Retail', 'ICT', 'Finance', 'Real Estate', 'Professional Service', 'Public Administration', 'Other Service']
df.index.names = ['Industry']
df

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Industry,,,,,,,,,,
Agriculture,37700,38200,36100,36100,36800,42700,40700,43200,40200,41100
Production,156700,149800,158600,154400,164200,173300,172300,162500,165100,165700
Construction,96600,93200,90000,91300,89300,97000,92600,102700,90800,101800
Retail,345400,344500,343100,347300,345100,337300,357700,360200,333500,347600
ICT,27800,27900,26400,27200,26900,35700,24000,34400,58900,31500
Finance,33800,29800,33200,31100,32400,32400,30800,31000,32100,35500
Real Estate,13500,14600,17600,18800,18000,22200,19100,22700,18200,25200
Professional Service,144800,145800,143600,137300,149900,152900,166200,161200,176400,187100
Public Administration,415600,418600,425600,421000,427000,427600,423200,418500,424500,434900


# 2. **Data Analysis**
### **2.1 Which industry employed highest and lowest workers over the period?**

In [12]:
df_total_jobs = df.apply(sum, axis=1).reset_index(name='total jobs provided').sort_values('total jobs provided', ascending=False)
df_total_jobs

,Industry,total jobs provided
8,Public Administration,4236500
3,Retail,3461700
1,Production,1622600
7,Professional Service,1565200
2,Construction,945300
9,Other Service,740800
0,Agriculture,392800
5,Finance,322100
4,ICT,320700
6,Real Estate,189900


In [13]:
fig = px.bar(df_total_jobs, x='Industry', y='total jobs provided', color='Industry')
fig.show()

From the above chart we can infer that over the period of 10 years, Public Administration was the largest employer with producing total 4.23 Million Jobs and Real Estate industry producing least number of jobs i.e. 190K.

### **2.2 Which industry has the highest and lowest overall growth over the period?**


In [0]:
'''
Here we define 2 functions to calculate overall growth in percentage as well as Absolute growth.
'''

def growth_pct(row):
    return round(((row['2018'] - row['2009'])/row['2009'])*100,2)
def growth_abs(row):
    return (row['2018'] - row['2009'])

In [15]:
df_growth_pct = df.apply(growth_pct, axis=1).reset_index(name='Growth in %').sort_values('Growth in %', ascending=False)
df_growth_abs = df.apply(growth_abs, axis=1).reset_index(name='Absolute Growth').sort_values('Absolute Growth', ascending=False)
print(df_growth_pct)
print('\n')
print(df_growth_abs)

                Industry  Growth in %
6            Real Estate        86.67
7   Professional Service        29.21
9          Other Service        27.41
4                    ICT        13.31
0            Agriculture         9.02
1             Production         5.74
2           Construction         5.38
5                Finance         5.03
8  Public Administration         4.64
3                 Retail         0.64


                Industry  Absolute Growth
7   Professional Service            42300
8  Public Administration            19300
9          Other Service            17600
6            Real Estate            11700
1             Production             9000
2           Construction             5200
4                    ICT             3700
0            Agriculture             3400
3                 Retail             2200
5                Finance             1700


In [16]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1,cols=2)
fig.add_bar(x=df_growth_abs['Industry'], y=df_growth_abs['Absolute Growth'], row=1, col=1)
fig.add_bar(x=df_growth_pct['Industry'], y=df_growth_pct['Growth in %'], row=1, col=2)
fig.update_layout(showlegend=True)
fig.data[0].name = 'Absolute Growth'
fig.data[1].name = 'Growth in %'
fig.show()

From the above bar chart we infer that Real Estate industry saw the highest overall growth with 86.67% rise in number of employees and Retail being the lowest with growth rate even less than 1%.
Whereas if we talk about absolute growth, Professional Services had an increase of 42k employees over the span of 10 years and finance saw a rise of 1.7k employees.

### **2.3 Which years are the best and worst performing year in relation to number of employment?**


In [17]:
df_emp_per_year = df.apply(sum).reset_index(name='total employment').sort_values('total employment', ascending=False)
df_emp_per_year

,index,total employment
9,2018,1452200
8,2017,1422900
7,2016,1408800
6,2015,1403800
5,2014,1394400
4,2013,1365100
2,2011,1346600
3,2012,1337300
0,2009,1336100
1,2010,1330400


In [18]:
fig = px.bar(df_emp_per_year, x='index', y='total employment', color='index')
fig.show()

From the visualization of total employment produced for each year we conclude that in the year 2018, we saw most number of new jobs being created i.e. 1.45M and in the year 2010 least number of jobs were created. However we only see marginal difference among the years i.e. there's no notable rise or fall in numbers.

# 3. **Visual Analysis using Plotly Express**

In [0]:
df_tp = df.T
df_tp = df_tp.reset_index()
df_tp_melt = pd.melt(df_tp, id_vars=['index'])


In [21]:
fig = px.line(df_tp_melt, x='index', y='value', color='Industry')
fig.show()

# 4. **Correlation**
### **4.1 Taking average employment number for each industry over the period, show and identify the highest and lowest correlated industries.**

In [22]:
df_t_corr_mat = df_tp.corr(method='pearson')
df_t_corr_mat

Industry,Agriculture,Production,Construction,Retail,ICT,Finance,Real Estate,Professional Service,Public Administration,Other Service
Industry,,,,,,,,,,
Agriculture,1.000000,0.646564,0.727019,0.227534,0.378069,-0.005239,0.668110,0.636513,0.194875,0.333264
Production,0.646564,1.000000,0.188308,0.028424,0.231710,0.225092,0.604355,0.560187,0.546657,0.577920
Construction,0.727019,0.188308,1.000000,0.414382,0.010326,0.309157,0.598064,0.440657,0.080268,-0.030656
Retail,0.227534,0.028424,0.414382,1.000000,-0.551669,-0.252533,0.232011,0.045887,-0.258487,-0.156339
ICT,0.378069,0.231710,0.010326,-0.551669,1.000000,0.042842,0.154186,0.503172,0.122228,0.543306
Finance,-0.005239,0.225092,0.309157,-0.252533,0.042842,1.000000,0.316411,0.388880,0.589635,0.242434
Real Estate,0.668110,0.604355,0.598064,0.232011,0.154186,0.316411,1.000000,0.622777,0.684273,0.617589
Professional Service,0.636513,0.560187,0.440657,0.045887,0.503172,0.388880,0.622777,1.000000,0.584972,0.800549
Public Administration,0.194875,0.546657,0.080268,-0.258487,0.122228,0.589635,0.684273,0.584972,1.000000,0.739090


In [0]:
'''
This block of code rounds the coefficient values for each industry upto 2 decimal points so that
it could be nicely displayed in the heatmap.
'''
np_round_corrmat = []
for x in df_t_corr_mat.to_numpy():
    row = []
    for y in x:
        row.append(round(y,2))
    np_round_corrmat.append(row)

In [24]:
import plotly.figure_factory as ff
fig = ff.create_annotated_heatmap(np_round_corrmat, x=list(df.index), y=list(df.index))
fig.show()

From the above data we conclude that Professional Service and Other Service industries are the most correlated as their correlation coefficient is `0.8` and ICT and retail are most negatively correlated industries with their correlation coefficient being `-0.55`


### **4.2 Make a year wise correlation for each industry. Does the aforementioned industries are also correlated over the each year? Explain your answer.**

# **5. Clustering**
### **5.1 Using the best and worst performing year column’s employment data (2.3) undertake a K means clustering analysis (K=2 & 3) and identify industries cluster together. Write your own interpretation (~100 words).**

In [25]:
df_best_worst_year = df[['2018','2010']]
series_index = df_best_worst_year.index
df_best_worst_year

,2018,2010
Industry,,
Agriculture,41100,38200
Production,165700,149800
Construction,101800,93200
Retail,347600,344500
ICT,31500,27900
Finance,35500,29800
Real Estate,25200,14600
Professional Service,187100,145800
Public Administration,434900,418600


In [0]:
kmeans_2 = KMeans(n_clusters=2)
kmeans_2.fit(df_best_worst_year)
labels = kmeans_2.predict(df_best_worst_year)
centroids = kmeans_2.cluster_centers_

In [27]:
fig = px.scatter(df_best_worst_year, '2018', '2010', color=labels.astype(str), color_continuous_scale=None, text=df_best_worst_year.index)
fig.add_scatter(x=centroids.T[0], y=centroids.T[1], mode='markers').update_traces(marker=dict(size=12), textposition='top center')
fig.update_layout(title=dict(text='K-means clustering with K=2'))
fig.data[-1].name = 'Centroids'
fig.data[-1].opacity = 0.5
fig.data[-1].marker.size = 40
fig.data[-1].marker.color = 'Black'
fig.data[0].name = 'Cluster-1'
fig.data[1].name = 'Cluster-2'
fig.show()

In [0]:
kmeans_3 = KMeans(n_clusters=3)
kmeans_3.fit(df_best_worst_year)
label_k3 = kmeans_3.predict(df_best_worst_year)
centroids_k3 = kmeans_3.cluster_centers_

In [29]:
fig = px.scatter(df_best_worst_year, '2018', '2010',color=label_k3.astype(str),text=series_index).update_traces(marker = dict(size=12), textposition='top center')
fig.add_scatter(x=centroids_k3.T[0],y=centroids_k3.T[1], mode='markers')
fig.data[0].name = 'Cluster-1'
fig.data[1].name = 'Cluster-2'
fig.data[2].name = 'Cluster-3'
fig.data[-1].name = 'Centroids' 
fig.data[-1].marker.color = 'Black'
fig.data[-1].marker.opacity = 0.5
fig.data[-1].marker.size = 40
fig.update_layout(title = dict(text='K-means clustering with K=3'))
fig

When performing k-means clustering with 2 clusters we find that industries such as `Finance, Real-Estate, Agriculture, ICT, Other Services, Construction, Production and Professional Service` in cluster reside in cluster-1 which has employed lesser number of employees in both 2010 and 2018 whereas only 2 industries i.e. `Retail and Public Administration` are in cluster-2 which characterises with higher number of employment in both years.
In k-means clustering with 3 clusters we conclude that cluster-1 includes `Real Estate, ICT, Finance, Agriculture, Other Services, Construction` industries where employment is low, cluster-2 incorporates `Production and Professional Service` industries whose new employment is satisfactory and cluster-3 have `Retail and Public Administration` which employ highest number of people.

### **5.2 Using the same dataset (best & worst performing) create a hierarchical cluster. Compare the cluster with k means clusters.**

In [0]:
from sklearn.cluster import AgglomerativeClustering

In [0]:
ndarray_best_worst_year = df_best_worst_year.to_numpy()

In [0]:
agg_cluster = AgglomerativeClustering()
label_agg = agg_cluster.fit_predict(ndarray_best_worst_year)

In [33]:
fig = px.scatter(df_best_worst_year, x='2018', y='2010', color=label_agg.astype(str), text=df_best_worst_year.index).update_traces(marker = dict(size=12), textposition='top center')
fig.data[0].name = 'Cluster-1'
fig.data[1].name = 'Cluster-2'
fig.update_layout(title = dict(text='Hierarchial Clustering with K=2'))
fig

In [0]:
agg_cluster_k3 = AgglomerativeClustering(n_clusters=3)
label_ag_k3 = agg_cluster_k3.fit_predict(ndarray_best_worst_year)

In [35]:
fig = px.scatter(df_best_worst_year, x='2018', y='2010', text=df.index, color=label_ag_k3.astype(str)).update_traces(textposition='top center', marker=dict(size=12))
fig.data[0].name = 'Cluster-1'
fig.data[1].name = 'Cluster-2'
fig.data[2].name = 'Cluster-3'
fig.show()

We conclude from the above visualization that for our best and worst year data there is no difference between k-means and hierarchial clustering.
Both processes yield the same result.

# 6. **Discussion**
### **Provide a brief discussion (~ 300 words) on employment landscape of Wales based on the employment data analysis results.**

In order to arrive at the results we had to apply all our knowledge of data pre-processing, data visualization, data analysis by applying clustering based machine learning algorithms.
After doing the aforementioned processes we gained some valuable insights on the employment landscape of Wales region, which are as follows.
- Over the course of 10 years Public Administration Industry employed the max number of employees i.e 4.2M and Real Estate employed the lowest i.e. 190K
- Even after being the employer to lowest number of people Real-Estate saw approx 87% growth over 10 years period which is highest among all industries.
- Professional Services saw the highest increase in employment over the 10 years span with an increase of 42K employees which is a good 29% growth.
- There has been a marginal increase at the overall employment provided by all the industries combined each year, with no steep rise or fall recorded. 
- 122K more people were employed in 2018 which is the highest among all years against 2010 in which lowest people were employed.
- In terms of correlation, most highly correlated countries are Professional Services and Other Services with their correlational coefficient at .8, whereas ICT and retail are most negatively correlated countries with their coefficient being equal to -0.55.
- After performing clustering on yearly employment data we infer that it is ideally divided into 3 clusters with their employment numbers resorting to poor, satisfactory and good.
- Public Administration has consistently been the highest employer year after year after ehich comes retail.
In a nutshell, we conclude that Public Administration and retail are the highest employers in Wales region while other industries have been also performing consistently among which Finance has seen the highest growth.